In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import duckdb

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [2]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
201219,CA,F,2012,Devorah,5
141733,CA,F,1997,Madelynne,5
238959,CA,F,2022,Niah,6
368478,CA,M,2009,Sammuel,11
292925,CA,M,1977,Justin,1266


## Goal 1: Find the most popular baby name in California in 2018

In [25]:
duckdb.sql(r"SELECT Name FROM babynames WHERE Year = 2018 ORDER BY Count DESC LIMIT 1").df()

,Name
0,Emma


## Goal 2: Baby names that start with j. 

In [24]:
duckdb.sql(r"SELECT DISTINCT Name FROM babynames WHERE Name LIKE 'J%'").df()

,Name
0,Jennine
1,Jessilyn
2,Juliane
3,Julyana
4,Jillian
...,...
1573,Joannie
1574,Jullie
1575,Jeanny
1576,Jennett


## Goal 3: Sort names by their length.

Approach 1a: Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

Approach 1b: Same as 1a, but use str.len() to generate the lengths of the strings.

Approach 2: Generate an index that is in the order we want. Pass that index to loc.

In [ ]:
babynames.loc[babynames["Name"].str.len().sort_values().index].head(5)

How does this work exactly? Let's break it into pieces.

In [ ]:
lengths_sorted_by_length = babynames["Name"].str.len().sort_values()
lengths_sorted_by_length.head(5)

In [ ]:
index_sorted_by_length = lengths_sorted_by_length.index
index_sorted_by_length

In [ ]:
# now pass the index to loc. This is yet another way 
# that loc can be used that we did not discuss in lecture!
babynames.loc[index_sorted_by_length].head(5)

In [21]:
# My approach: using SQL?
babynames["Name_Length"] = babynames["Name"].map(len)
index_sorted_by_length_sql = duckdb.sql(r"SELECT * FROM babynames ORDER BY Name_Length").df().drop("Name_Length", axis=1)
index_sorted_by_length_sql

,State,Sex,Year,Name,Count
0,CA,F,1987,Pa,33
1,CA,F,1987,Ka,17
2,CA,F,1987,My,16
3,CA,F,1987,An,11
4,CA,F,1987,Vi,10
...,...,...,...,...,...
407423,CA,M,1996,Franciscojavier,8
407424,CA,M,1997,Franciscojavier,5
407425,CA,M,1997,Ryanchristopher,5
407426,CA,M,1998,Franciscojavier,6
